# Hosting Libcom models on Sagemaker using BYOC (bring-your-own-container)
in this bootbook, we demonstrate how to host libcom (the library of image composition) on SageMaker real-time endpoint using BYOC (Bring-your-own-container)

## Prepare pre-requisites tools

#### install and update sagemaker tools

In [ ]:
%pip install --upgrade pip --quiet
%pip install sagemaker boto3 awscli --upgrade --quiet

#### Initialize the variables for SageMaker default bucket, role, and AWS account ID, and current AWS region.

In [ ]:
import sagemaker
import boto3

account_id = boto3.client("sts").get_caller_identity().get("Account")
sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
role = sagemaker.get_execution_role()
region_name = boto3.session.Session().region_name

#### Execute the script - download_pretrained_model.sh to download pretrained models we will use

In [ ]:
# download models
!chmod +x download_pretrained_model.sh && bash download_pretrained_model.sh

#### Execute the script - build_and_push.sh to build Docker images for SageMaker endpoint.

In [ ]:
# build containers used by sagemaker
!chmod +x build_and_push.sh && bash build_and_push.sh $region_name

#### Upload the dummy file to S3 to meet the requirement of SageMaker Endpoint for model data.

In [ ]:
s3_model_prefix = "libcom"

!touch dummy
!rm -f model.tar.gz
!tar czvf model.tar.gz dummy
s3_model_artifact = sagemaker_session.upload_data("model.tar.gz", bucket, s3_model_prefix)
print(f"S3 Code or Model tar uploaded to --- > {s3_model_artifact}")
!rm -f dummy

## Deploy Sagemaker Real-time Endpoint

#### Define the model

In [ ]:
from sagemaker import Model, serializers, deserializers
inference_algorithm_name='libcom'
image_uri_inference = (
    f"{account_id}.dkr.ecr.{region_name}.amazonaws.com/{inference_algorithm_name}:latest"
)
print(image_uri_inference)

model_name='libcom'
model = Model(
    name=model_name,
    image_uri=image_uri_inference, 
    model_data=s3_model_artifact, 
    role=role)

#### deploy the endpoint

In [ ]:
instance_type = "ml.g5.2xlarge"  # "ml.g5.2xlarge" - #single GPU. 

endpoint_name = 'libcom'
model.deploy(
    image_uri=image_uri_inference,
    initial_instance_count=1,
    instance_type=instance_type,
    endpoint_name=endpoint_name,
    container_startup_health_check_timeout=900,
)

# our requests and responses will be in json format so we specify the serializer and the deserializer
predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=serializers.JSONSerializer(),
    deserializer=deserializers.JSONDeserializer()
)

## Uncomment below and Execute Clean up after your experiment finished

In [ ]:
# delete endpoint for cost saving
# endpoint_name = 'libcom'
# model_name='libcom'
# import boto3
# sagemaker_client = boto3.client("sagemaker")

# Delete the endpoint
# sagemaker_client.delete_endpoint(EndpointName=endpoint_name)

# Delete the endpoint configuration
# sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_name)

# Delete the model
# sagemaker_client.delete_model(ModelName=model_name)